# 🌱 Garden Groups - Fence Cost Calculation

The puzzle involves calculating the total price of fencing around regions of garden plots based on their **area** and **perimeter**.

## Problem Description

### Garden Layout
- Each garden plot grows a single type of plant, represented by a letter.
- Adjacent plots of the same plant (horizontally or vertically) form a **region**.

### Key Metrics
- **Area**: Total number of plots in a region.
- **Perimeter**: Total exposed sides of the region.

### Fence Pricing
- **Price per Region**: `area × perimeter`
- **Total Price**: Sum of all region prices.

### Bulk Discount
For large orders, **perimeter** is replaced by the number of **sides**:
- Each straight section of the fence counts as a side.
- New **price per region**: `area × sides`
___


In [ ]:
def read_map(file_path):
    """Reads the garden map from the input file."""
    with open(file_path, 'r') as file:
        return [list(line.strip()) for line in file.readlines()]

def flood_fill(grid, x, y, visited, plant_type):
    """Performs flood-fill to find a region and calculates its area and perimeter."""
    rows, cols = len(grid), len(grid[0])
    stack = [(x, y)]
    area = 0
    perimeter = 0
    visited.add((x, y))

    # Directions for movement: up, down, left, right
    directions = [(-1, 0), (1, 0), (0, -1), (0, 1)]

    while stack:
        cx, cy = stack.pop()
        area += 1

        for dx, dy in directions:
            nx, ny = cx + dx, cy + dy
            if 0 <= nx < rows and 0 <= ny < cols:
                if grid[nx][ny] == plant_type:
                    if (nx, ny) not in visited:
                        visited.add((nx, ny))
                        stack.append((nx, ny))
                else:
                    # Edge of the region
                    perimeter += 1
            else:
                # Edge of the grid
                perimeter += 1

    return area, perimeter

def calculate_fencing_cost(grid):
    """Calculates the total fencing cost for the garden."""
    rows, cols = len(grid), len(grid[0])
    visited = set()
    total_cost = 0

    for x in range(rows):
        for y in range(cols):
            if (x, y) not in visited:
                # Start a new region
                plant_type = grid[x][y]
                area, perimeter = flood_fill(grid, x, y, visited, plant_type)
                total_cost += area * perimeter

    return total_cost

def main():
    # Read the garden map from the input file
    file_path = "input.txt"
    garden_map = read_map(file_path)

    # Calculate the total fencing cost
    total_cost = calculate_fencing_cost(garden_map)

    # Output the result
    print("Total fencing cost:", total_cost)

if __name__ == "__main__":
    main()


Total fencing cost: 1361494


# Part 2

In [ ]:
def find_regions(grid):
    rows, cols = len(grid), len(grid[0])
    visited = set()
    regions = []

    def dfs(r, c, plant_type):
        if (r, c) in visited or r < 0 or c < 0 or r >= rows or c >= cols or grid[r][c] != plant_type:
            return set()

        cells = {(r, c)}
        visited.add((r, c))

        for dr, dc in [(0, 1), (1, 0), (0, -1), (-1, 0)]:
            cells.update(dfs(r + dr, c + dc, plant_type))

        return cells

    for r in range(rows):
        for c in range(cols):
            if (r, c) not in visited:
                region = dfs(r, c, grid[r][c])
                if region:
                    regions.append(region)

    return regions

def count_sides(region, grid):
    rows, cols = len(grid), len(grid[0])
    sides = 0
    visited_edges = set()

    def trace_line(r, c, dr, dc):
        nr, nc = r + dr, c + dc
        return (nr < 0 or nc < 0 or nr >= rows or nc >= cols or (nr, nc) not in region)

    # Count horizontal lines
    for r, c in region:
        # Check up
        if (r, c, 'u') not in visited_edges and trace_line(r, c, -1, 0):
            start_c = c
            while (r, start_c-1) in region and trace_line(r, start_c-1, -1, 0):
                start_c -= 1
            end_c = c
            while (r, end_c+1) in region and trace_line(r, end_c+1, -1, 0):
                end_c += 1
            sides += 1
            for i in range(start_c, end_c+1):
                visited_edges.add((r, i, 'u'))

        # Check down
        if (r, c, 'd') not in visited_edges and trace_line(r, c, 1, 0):
            start_c = c
            while (r, start_c-1) in region and trace_line(r, start_c-1, 1, 0):
                start_c -= 1
            end_c = c
            while (r, end_c+1) in region and trace_line(r, end_c+1, 1, 0):
                end_c += 1
            sides += 1
            for i in range(start_c, end_c+1):
                visited_edges.add((r, i, 'd'))

    # Count vertical lines
    for r, c in region:
        # Check left
        if (r, c, 'l') not in visited_edges and trace_line(r, c, 0, -1):
            start_r = r
            while (start_r-1, c) in region and trace_line(start_r-1, c, 0, -1):
                start_r -= 1
            end_r = r
            while (end_r+1, c) in region and trace_line(end_r+1, c, 0, -1):
                end_r += 1
            sides += 1
            for i in range(start_r, end_r+1):
                visited_edges.add((i, c, 'l'))

        # Check right
        if (r, c, 'r') not in visited_edges and trace_line(r, c, 0, 1):
            start_r = r
            while (start_r-1, c) in region and trace_line(start_r-1, c, 0, 1):
                start_r -= 1
            end_r = r
            while (end_r+1, c) in region and trace_line(end_r+1, c, 0, 1):
                end_r += 1
            sides += 1
            for i in range(start_r, end_r+1):
                visited_edges.add((i, c, 'r'))

    return sides

def solve_part2(input_text):
    grid = [list(line.strip()) for line in input_text.strip().split('\n')]
    regions = find_regions(grid)

    total_price = 0
    for region in regions:
        area = len(region)
        num_sides = count_sides(region, grid)
        price = area * num_sides
        total_price += price

    return total_price

with open('input.txt', 'r') as f:
    input_text = f.read()

result = solve_part2(input_text)
print(result)

830516



## Objectives
1. Calculate the **total price** of fencing all regions using both methods.
2. Implement efficient algorithms to handle larger maps.

**Puzzle Completion:** Both parts solved, earning two gold stars! 🌟🌟
